<a href="https://colab.research.google.com/github/ryuqae/EntityRelation/blob/main/Preprocessing%20and%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project: 2021년 국립국어원 인공지능 언어능력 평가

- [2021년 국립국어원 인공지능 언어능력 평가](https://corpus.korean.go.kr/task/taskList.do?taskId=1&clCd=END_TASK&subMenuId=sub01) 는 9월 1일부터 시작하여 11월 1일까지 마감된 [네 가지 과제에](https://corpus.korean.go.kr/task/taskDownload.do?taskId=1&clCd=END_TASK&subMenuId=sub02) 대한 언어능력 평가 대회
- 여기서 제시된 과제를 그대로 수행하여 그 결과를 [최종 선정된 결과들](https://corpus.korean.go.kr/task/taskLeaderBoard.do?taskId=4&clCd=END_TASK&subMenuId=sub04)과 비교할 수 있도록 수행
- 아직 테스트 셋의 정답이 공식적으로 공개되고 있지 않아, 네 가지 과제의 자료에서 evaluation dataset으로 가지고 성능을 비교할 계획
- 기말 발표전까지 정답셋이 공개될 경우 이 정답셋을 가지고 성능 검증
- Transformers 기반 방법론, 신경망 등 각자 생각한 방법대로 구현 가능
- 현재 대회기간이 종료되어 자료가 다운로드 가능하지 않으니 첨부된 자료 참조
- 개인적으로 하거나 최대 두명까지 그룹 허용. 
- 이 노트북 화일에 이름을 변경하여 작업하고 제출. 제출시 화일명을 FinalProject_[DS또는 CL]_학과_이름.ipynb
- 마감 12월 6일(월) 23:59분까지.
- 12월 7일, 9일 기말 발표 presentation 예정

## 리더보드

- 최종발표전까지 각조는 각 태스크별 실행성능을 **시도된 여러 방법의 결과들을 지속적으로**  [리더보드](https://docs.google.com/spreadsheets/d/1-uenfp5GolpY2Gf0TsFbODvj585IIiFKp9fvYxcfgkY/edit#gid=0)에 해당 팀명(구성원 이름 포함)을 입력하여 공개하여야 함. 
- 최종 마감일에 이 순위와 실제 제출한 프로그램의 수행 결과를 비교하여 성능을 확인

In [ ]:
!pip install transformers

from functools import partial
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import AutoTokenizer, RobertaPreTrainedModel, AutoConfig, RobertaModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer
import torch.nn.functional as F

import torchtext
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

import os
import numpy as np
import random
import math
import time
import re
import logging

print("\n-- Packages Version --")
# print(f"{'NLTK':<10} | {nltk.__version__:>7}")
print(f"{'PyTorch':<10} | {torch.__version__:>7}")
print(f"{'TorchText':<10} | {torchtext.__version__:>7}")

print("\n-- CUDA Availability --")
# Set and check available cuda device

GPU_NUM = 0
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

if device.type=='cuda':
    print(f"{torch.cuda.get_device_properties(device).name}\nDevice Num: {torch.cuda.current_device()}")
else:
    print('Warning: cuda is not available')
    
    
def init_logger():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

def set_seed(SEED=2934):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


-- Packages Version --
PyTorch    | 1.10.0+cu111
TorchText  |  0.11.0

-- CUDA Availability --
Tesla P100-PCIE-16GB
Device Num: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Mon Dec  6 12:58:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import AutoModel, AutoTokenizer, RobertaPreTrainedModel, AutoConfig, RobertaModel

ADDITIONAL_SPECIAL_TOKENS = ["<e1>", "</e1>", "<e2>", "</e2>"]
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", return_token_type_ids=False)
tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})

4

In [ ]:
from glob import glob

data_dir = glob('*.tsv')

In [ ]:
import pandas as pd

def load_data(dataset_dir, mode = 'train'):
    dataset = pd.read_csv(dataset_dir, delimiter='\t')
    if mode == 'test':
        dataset["ANSWER"] = [0] * len(dataset)
    dataset["ANSWER"] = dataset["ANSWER"].astype(int)
    return dataset

In [ ]:
set_seed()

In [ ]:
class HomonymDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def convert_sentence_to_features(dataset, tokenizer, max_len):
    # from distutils.util import strtobool

    max_seq_len=max_len
    pad_token=tokenizer.pad_token_id
    add_sep_token=False
    mask_padding_with_zero=True
    
    all_input_ids = []
    all_attention_mask = []
    all_e1_mask=[]
    all_e2_mask=[]
    all_label=[]
    m_len=0
        
    err = 0

    for idx in tqdm(range(len(dataset))):
        sentence = '[CLS]' + dataset['SENTENCE1'][idx][:dataset['start_s1'][idx]] \
        + '<e1>' + dataset['SENTENCE1'][idx][dataset['start_s1'][idx]:dataset['end_s1'][idx]] \
        + '</e1>' + dataset['SENTENCE1'][idx][dataset['end_s1'][idx]:] + '[SEP]' + dataset['SENTENCE2'][idx][:dataset['start_s2'][idx]] \
        + ' <e2> ' + dataset['SENTENCE2'][idx][dataset['start_s2'][idx]:dataset['end_s2'][idx]] \
        + ' </e2> ' + dataset['SENTENCE2'][idx][dataset['end_s2'][idx]:] + '[SEP]'
        
        token = tokenizer.tokenize(sentence)
        m_len = max(m_len, len(token))
        e11_p = token.index("<e1>")  # the start position of entity1
        e12_p = token.index("</e1>")  # the end position of entity1
        e21_p = token.index("<e2>")  # the start position of entity2
        e22_p = token.index("</e2>")  # the end position of entity2

        token[e11_p] = "$"
        token[e12_p] = "$"
        token[e21_p] = "#"
        token[e22_p] = "#"

        special_tokens_count = 1

        # masks for entity
        if len(token) < max_seq_len - special_tokens_count:
            input_ids = tokenizer.convert_tokens_to_ids(token)
            attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            padding_length = max_seq_len - len(input_ids)
            input_ids = input_ids + ([pad_token] * padding_length)
            attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)

            e1_mask = [0] * len(attention_mask)
            e2_mask = [0] * len(attention_mask)

            for i in range(e11_p, e12_p + 1):
                e1_mask[i] = 1
            
            for j in range(e21_p, e22_p + 1):
                e2_mask[j] = 1

            assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
            assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(
                len(attention_mask), max_seq_len
            )

            all_input_ids.append(input_ids)
            all_attention_mask.append(attention_mask)
            all_e1_mask.append(e1_mask)
            all_e2_mask.append(e2_mask)
            all_label.append(dataset['ANSWER'][idx])

    all_features = {
        'input_ids' : torch.tensor(all_input_ids),
        'attention_mask' : torch.tensor(all_attention_mask),
        'e1_mask' : torch.tensor(all_e1_mask),
        'e2_mask' : torch.tensor(all_e2_mask)
    }

    return HomonymDataset(all_features, all_label)


In [ ]:
train_raw = load_data(data_dir[0])

# Model

In [ ]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout=0.0, use_activation=True):
        super().__init__()
        self.use_activation = use_activation
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(self.input_dim, self.output_dim)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.dropout(x)
        if self.use_activation:
            x = self.tanh(x)
        x = self.fc1(x)
        return x

In [ ]:
class HomonymNet(RobertaPreTrainedModel):
    def __init__(self, checkpoint, config, dropout_rate):
        super().__init__(config)
        self.model = AutoModel.from_pretrained(checkpoint, config=config)
        self.num_labels = config.num_labels

        self.cls_fclayer = FCLayer(config.hidden_size, config.hidden_size, dropout_rate)
        self.e1_fclayer = FCLayer(config.hidden_size, config.hidden_size, dropout_rate)
        self.e2_fclayer = FCLayer(config.hidden_size, config.hidden_size, dropout_rate)
        self.label_classifier = FCLayer(
            config.hidden_size * 3, # concat cls, e1, e2 output
            config.num_labels,
            dropout_rate,
            use_activation=False,
        )
    
    @staticmethod
    def entity_hidden_average(hidden_output, entity_mask):

        unsq_entity_mask = entity_mask.unsqueeze(1)
        length_tensor = (entity_mask != 0).sum(dim=1).unsqueeze(1)

        entity_sum = torch.bmm(unsq_entity_mask.float(), hidden_output).squeeze(1)
        entity_average = entity_sum.float() / length_tensor.float()  # broadcasting

        return entity_average


    def forward(self, input_ids, attention_mask, e1_mask, e2_mask, labels):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]

        sentence_representation = self.cls_fclayer(outputs.pooler_output)

        e1_hidden = self.entity_hidden_average(sequence_output, e1_mask)
        e2_hidden = self.entity_hidden_average(sequence_output, e2_mask)

        e1_hidden = self.e1_fclayer(e1_hidden)
        e2_hidden = self.e2_fclayer(e2_hidden)

        concat_hidden = torch.cat([sentence_representation, e1_hidden, e2_hidden], dim=-1)
        logits = self.label_classifier(concat_hidden)
        outputs = (logits,) + outputs[2:]

        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
        outputs = (loss,) + outputs

        return outputs

In [ ]:
# test = convert_sentence_to_features(train_raw[10:13], tokenizer, 100)

# Train

In [ ]:
num_train_epochs = 10
adam_learning_rate = 1e-5
adam_epsilon = 1e-8
weight_decay = 1e-2

gradient_accumulation_steps = 2

max_len = 200
batch_size = 16
eval_batch_size = 16

In [ ]:
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    return acc_and_f1(preds, labels)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def acc_and_f1(preds, labels, average="macro"):
    acc = simple_accuracy(preds, labels)
    return {
        "acc": acc,
    }

def init_logger():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

In [ ]:
config = AutoConfig.from_pretrained(
    "klue/roberta-large",
    num_labels=2
)

model = HomonymNet(
    "klue/roberta-large",
    config=config,
    dropout_rate=0.1
)

model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=adam_learning_rate,
    eps=adam_epsilon,
)


# train_dataloader = DataLoader(train_dataset, batch_size=16)
# total_steps = len(train_dataloader) * num_train_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps = 0,
#                                             num_training_steps = total_steps)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [ ]:
model.zero_grad()

In [ ]:
test_raw = load_data("/content/NIKL_SKT_WiC_Dev.tsv")
test_dataset = convert_sentence_to_features(test_raw, tokenizer, max_len=max_len)

base_dir = '/content/drive/MyDrive/NLP_FinalProject/'

def save_checkpoint(model, model_dir):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(model_dir)


def evaluate(model, fold_id, best_score, mode='valid'):
    if mode=='valid':
        test_dataloader = DataLoader(valid_dataset, batch_size=eval_batch_size)
    elif mode=='test':
        test_dataloader = DataLoader(test_dataset, batch_size=eval_batch_size)

    predict = None
    eval_loss = 0.0
    nb_eval_steps = 0

    model.eval()

    for batch in tqdm(test_dataloader, desc="Evaluating"):
        batch = tuple(batch[t].to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids":batch[0],
                "attention_mask":batch[1],
                "e1_mask":batch[2],
                "e2_mask":batch[3],
                "labels":batch[4]
            }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps+=1

        if predict is None:
            predict = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            predict = np.append(predict, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
        # print(list(zip(predict_label, out_label_ids)))


    predict_label = np.argmax(predict, axis=1)
    result = compute_metrics(predict_label, out_label_ids)

    eval_loss = eval_loss / nb_eval_steps
    results = {"loss": eval_loss}
    results.update(result)

    if mode =='valid':
        if result['acc'] > best_score:
            save_checkpoint(model, f"{base_dir}model_fold_{fold_id}")
            best_score = result['acc']
            print(f"Saved new best model - acc : {best_score}")

    return results, best_score

  0%|          | 0/1166 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, random_state=2934, shuffle=True)

best_score=0

for fold_id, (train_ids, valid_ids) in tqdm(enumerate(kfold.split(train_raw))):

    train_dataset = convert_sentence_to_features(train_raw.iloc[train_ids].reset_index(), tokenizer, max_len=max_len)
    valid_dataset = convert_sentence_to_features(train_raw.iloc[valid_ids].reset_index(), tokenizer, max_len=max_len)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    total_steps = len(train_dataloader) * num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print(f"FOLD {fold_id+1} : Split train dataset to train vs valid")

    train_loss = 0.0
    fold_best_score = 0

    for epoch_step in tqdm(range(num_train_epochs), desc="Epoch"):
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            model.train()
            batch = tuple(batch[t].to(device) for t in batch)
            inputs = {
                "input_ids":batch[0],
                "attention_mask":batch[1],
                "e1_mask":batch[2],
                "e2_mask":batch[3],
                "labels":batch[4]
            }

            outputs = model(**inputs)
            loss = outputs[0]
            loss.backward()

            train_loss += loss.item()

            optimizer.step()
            scheduler.step()
            model.zero_grad()

        valid_loss, fold_best_score = evaluate(model, fold_id+1, fold_best_score, 'valid')
        test_loss, _ = evaluate(model, fold_id+1, fold_best_score, 'test')
            
        print(f"============================= Epoch #{epoch_step+1} =============================")
        print(f" - train: {train_loss}")
        print(f" - valid: {valid_loss}")
        print(f" - test : {test_loss}")
    
    print(f"FOLD {fold_id+1} Best Validation Score : {fold_best_score}")

0it [00:00, ?it/s]

  0%|          | 0/6198 [00:00<?, ?it/s]

  0%|          | 0/1550 [00:00<?, ?it/s]

FOLD 1 : Split train dataset to train vs valid


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch #8=============================
 - train: 27.8457896581549
 - valid: {'loss': 0.009496904418446105, 'acc': 0.9980607627666451}
 - test : {'loss': 0.5904141553148218, 'acc': 0.9048027444253859}


Iteration:   0%|          | 0/388 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/97 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/73 [00:00<?, ?it/s]

Epoch #9=============================
 - train: 27.929937062446697
 - valid: {'loss': 0.009420584432631322, 'acc': 0.9980607627666451}
 - test : {'loss': 0.5948210134378539, 'acc': 0.9039451114922813}
FOLD 5 Best Validation Score : 0.9987071751777634
